In [1]:
import pandas as pd
import numpy as np
import json
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import time
from googleapiclient.errors import HttpError

# --- Display Options ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

In [2]:
# --- Authentication and Setup ---
SERVICE_ACCOUNT_FILE = 'C:/Users/511232/Desktop/DSS/MERGING GOOGLESHEETS QUESTIONNAIRES/online questionnnaires/credentials_serviceaccount.json'
SCOPES = [
    'https://www.googleapis.com/auth/drive.readonly',
    'https://www.googleapis.com/auth/spreadsheets.readonly'
]
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=creds)
sheets_service = build('sheets', 'v4', credentials=creds)

# --- Global Variables ---
MAIN_FOLDER_ID = '1nZ8l69GkB6EQQPoiOF-6CEVx2q3vAhcU'
keywords = ['Health', 'Education', 'Housing', 'Population', 'Labor', 'Poverty']
results = []

In [3]:
# --- Helper Functions for Google Drive API ---
def execute_with_backoff(api_call):
    """Executes an API call with exponential backoff for handling rate limiting."""
    max_retries = 5
    base_delay = 1
    for n in range(max_retries):
        try:
            return api_call.execute()
        except (TimeoutError, HttpError) as e:
            if isinstance(e, HttpError) and e.resp.status not in [429, 500, 503]:
                print(f"A non-retriable HTTP error occurred: {e}")
                return None
            if n == max_retries - 1:
                print(f"Max retries reached. Failing with error: {e}")
                raise e
            delay = base_delay * (2 ** n) + np.random.uniform(0, 1)
            print(f"API Error (Status: {e.resp.status if isinstance(e, HttpError) else 'Timeout'}). Retrying in {delay:.2f} seconds...")
            time.sleep(delay)

In [4]:
def list_files_in_folder(folder_id, folder_path):
    """Recursively lists Google Sheets in a folder and its subfolders."""
    try:
        query_folders = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder'"
        request_folders = drive_service.files().list(q=query_folders, fields="files(id, name)")
        response_folders = execute_with_backoff(request_folders)
        if not response_folders: return

        for subfolder in response_folders.get('files', []):
            list_files_in_folder(subfolder['id'], folder_path + [subfolder['name']])

        query_files = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
        request_files = drive_service.files().list(q=query_files, fields="files(id, name)")
        response_files = execute_with_backoff(request_files)
        if not response_files: return

        for file in response_files.get('files', []):
            sheet_metadata = execute_with_backoff(sheets_service.spreadsheets().get(spreadsheetId=file['id']))
            if not sheet_metadata: continue

            for sheet in sheet_metadata.get('sheets', []):
                tab_name = sheet['properties']['title']
                if any(keyword.lower() in tab_name.lower() for keyword in keywords):
                    results.append({
                        'folder_path': '/'.join(folder_path),
                        'file_name': file['name'],
                        'tab_name': tab_name,
                        'gid': sheet['properties']['sheetId'],
                        'url': f"https://docs.google.com/spreadsheets/d/{file['id']}/edit#gid={sheet['properties']['sheetId']}"
                    })
    except Exception as e:
        print(f"An unexpected error occurred in list_files_in_folder: {e}")

def get_sheets_from_drive(root_folder_id, select_folders='all'):
    """Controller function to process folders in Google Drive."""
    main_folder_name = 'Main'
    if select_folders == 'all':
        list_files_in_folder(root_folder_id, [main_folder_name])
    elif isinstance(select_folders, list):
        query_country_folders = f"'{root_folder_id}' in parents and mimeType='application/vnd.google-apps.folder'"
        request_folders = drive_service.files().list(q=query_country_folders, fields="files(id, name)")
        response_folders = execute_with_backoff(request_folders)
        if not response_folders: return
        all_country_folders = response_folders.get('files', [])
        found_any = False
        for country_folder in all_country_folders:
            folder_name = country_folder['name']
            folder_id = country_folder['id']
            if folder_name in select_folders:
                found_any = True
                print(f"Found and processing folder: {folder_name}")
                list_files_in_folder(folder_id, [main_folder_name, folder_name])
        if not found_any:
            print("Did not find any of the specified folders.")


In [5]:
def process_df(df, theme):
    """
    Cleans, reshapes a dataframe, merges it with its source information, and adds a theme column.
    """
    df1 = df.dropna(how='all')
    try:
        source_row_index = df1[df1.isin(['المصدر','Source', 'source']).any(axis=1)].index[0]
    except IndexError:
        print("Warning: 'Source' keyword not found. The sheet might have an unexpected format.")
        return pd.DataFrame()

    source_col = df.columns[df1.isin(['المصدر','Source', 'source']).any(axis=0)][0]
    source_col_index = df1.columns.get_loc(source_col)
    
    main_df = df1.iloc[0:source_row_index-1].reset_index(drop=True)
    source_df = df1.iloc[source_row_index-1 :,source_col_index :].reset_index(drop=True)

    year_columns = [col for col in df.columns if str(col).strip().isdigit()]
    if not year_columns:
        print("Warning: No year columns found to melt.")
        return pd.DataFrame()

    first_year_col_index = main_df.columns.get_loc(year_columns[0])
    id_variables = list(main_df.columns[:first_year_col_index])

    df_main_long = pd.melt(main_df,
                          id_vars=id_variables,
                          value_vars=year_columns,
                          var_name='Year',
                          value_name='Value',
                          ignore_index=True)

    source_df_T = source_df.T.reset_index()
    cols = ['Year','Source']
    source_df_T = source_df_T.iloc[1:]
    source_df_T.columns = cols

    # Convert Year columns to the same type for a reliable merge
    df_main_long['Year'] = df_main_long['Year'].astype(str)
    source_df_T['Year'] = source_df_T['Year'].astype(str)
    merged_data = pd.merge(df_main_long, source_df_T, on='Year', how='left')

    merged_data['Theme'] = theme
    return merged_data

In [6]:
# --- Main Script Execution ---
get_sheets_from_drive(MAIN_FOLDER_ID, select_folders=['Algeria', 'Saudi Arabia'])
df_urls = pd.DataFrame(results)
if not df_urls.empty:
    df_urls['pandas_url'] = df_urls['url'].str.replace('/edit#gid=', '/export?format=csv&gid=')
    df_urls.to_csv('urls_gids.csv', index=False)
    print(f"URLs and GIDs saved to urls_gids.csv")

df_ar = []
df_en = []
if not df_urls.empty:
    for idx, row in df_urls.iterrows():
        theme = next((k for k in keywords if k.lower() in row['tab_name'].lower()), "Unknown")
        
        language_folder = 'English' if 'English' in row['folder_path'] else 'Arabic'
        print(f"Processing {language_folder} file: {row['file_name']} | Tab: {row['tab_name']}")
        try:
            df = pd.read_csv(row['pandas_url'])
            df_result = process_df(df, theme)
            if not df_result.empty:
                if language_folder == 'English':
                    df_en.append(df_result)
                else:
                    df_ar.append(df_result)
        except Exception as e:
            print(f"Could not process file {row['file_name']}. Error: {e}")

Found and processing folder: Saudi Arabia
Found and processing folder: Algeria
URLs and GIDs saved to urls_gids.csv
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_1_a
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_1_b
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_2_a
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_2_b
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_2_c
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_3
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_4
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_5
Processing Arabic file: Saudi Arabia_Education_Questionnaire | Tab: Education_6
Processing Arabic file: Saudi Arabia_Health_Questionnaire 2025 | Tab: Health_1_a
Processing Arabic file: Saudi Arabia_Health_Questionnaire 2025 | Tab: Hea

In [ ]:
# --- Step 1: Process the Arabic DataFrame ---
if df_ar:
    print("\n--- Consolidating Arabic DataFrames ---")
    merged_ar_df = pd.concat(df_ar, ignore_index=True)
    
    ind_correction = pd.read_excel('arabic to arabic indicator correction.xlsx')
    indicator_correction_dict = dict(zip(ind_correction['Indicators_originally_before_consolidation'], ind_correction['Indicator_mapping_consolidated']))
    
    merged_ar_df['Theme'] = merged_ar_df['Theme'].map({'Population':'السكان', 'Health':'الصحة','Education':'التعليم','Housing':'السكن','Labor':'العمالة','Poverty':'الفقر'})
    merged_ar_df.rename(columns={'Year': 'السنة', 'Value': 'العدد', 'Source': 'المصدر', 'Theme':'الفصل'}, inplace=True)
    
    if 'المؤشر' in merged_ar_df.columns:
        merged_ar_df['المؤشر'] = merged_ar_df['المؤشر'].str.strip()
        merged_ar_df['المؤشر'] = merged_ar_df['المؤشر'].map(indicator_correction_dict)

    # list of columns to fill with the placeholder to prevent overwriting them.
    cols_to_fill = [col for col in merged_ar_df.columns if col not in ['الدولة', 'السنة', 'العدد', 'المؤشر', 'الفصل', 'المصدر']]
    
    for col in cols_to_fill:
        merged_ar_df[col] = merged_ar_df[col].fillna('غير مطابق')
else:
    merged_ar_df = pd.DataFrame()

# --- Step 2: Process the English DataFrame ---
if df_en:
    print("\n--- Consolidating English DataFrames ---")
    merged_en_df = pd.concat(df_en, ignore_index=True)
    # Fill NaNs with 'Not Applicable'
    cols_to_fill_en = [col for col in merged_en_df.columns if col not in ['Country','Year','Value']]
    for col in cols_to_fill_en:
        merged_en_df[col] = merged_en_df[col].fillna('Not Applicable')
else:
    merged_en_df = pd.DataFrame()

# --- Step 3: Translate and Merge English data into Arabic data (if it exists) ---
if not merged_en_df.empty:
    print("\n--- Translating and Merging English Data ---")
    path='C:/Users/511232/Desktop/DSS/MERGING GOOGLESHEETS QUESTIONNAIRES/codes'
    df_translation=pd.read_excel(path+'/translation dict.xlsx')
    
    En_Ar_dictionary={}
    for dim in [d for d in df_translation['col_en'].unique() if d not in ['year', 'value', 'source']]:
        df_dim=df_translation[df_translation['col_en'].isin([dim.lower(), dim])].copy()
        En_Ar_dictionary.update(
            {dim:{'dim_values':dict(zip(df_dim['val_en'], df_dim['val_ar'])), 
                  'dim': {df_dim['col_en'].unique()[0]:df_dim['col_ar'].unique()[0]}}})

    Ar_En_dictionary={}
    for dim in [d for d in df_translation['col_ar'].unique() if d not in ['السنة', 'العدد', 'المصدر']]:
        df_dim=df_translation[df_translation['col_ar'].isin([dim.lower(), dim])].copy()
        Ar_En_dictionary.update(
            {dim:{'dim_values':dict(zip(df_dim['val_ar'], df_dim['val_en'])), 
                  'dim': {df_dim['col_ar'].unique()[0]:df_dim['col_en'].unique()[0]}}})

    class Translator:
        def __init__(self, translate_to, en_ar_dict=None,ar_en_dict=None):
            if translate_to.lower() not in ['english', 'arabic']:
                raise ValueError("Language must be 'english' or 'arabic'")
            self.translate_to = translate_to.lower()
            self.en_ar_dict = en_ar_dict if en_ar_dict is not None else En_Ar_dictionary
            self.ar_en_dict = ar_en_dict if ar_en_dict is not None else Ar_En_dictionary
        def translate(self, df):
            df_translated = df.copy()
            translation_dict = self.ar_en_dict if self.translate_to == 'english' else self.en_ar_dict
            for col, col_dict in translation_dict.items():
                if col in df_translated.columns:
                    new_col_name = list(col_dict['dim'].values())[0]  
                    df_translated[col] = df_translated[col].replace(col_dict['dim_values'])
                    df_translated.rename(columns=col_dict['dim'], inplace=True)
            return df_translated

    translator = Translator('arabic')
    en_df_translated = translator.translate(merged_en_df)
    
    final_df = pd.concat([merged_ar_df, en_df_translated], ignore_index=True)
else:
    final_df = merged_ar_df

# 4. Save the Final Output
if not final_df.empty:
    print("\nSaving final combined Arabic questionnaires file...")
    final_df.to_excel('arabic_questionnaires.xlsx', index=False)
else:
    print("\nNo data was processed to save.")

print('\nScript finished.')


--- Consolidating Arabic DataFrames ---

Saving final combined Arabic questionnaires file...


#### check schema with pandera and rapidfuzz to match with possible different column names

In [13]:
import pandas as pd
from rapidfuzz import process
import pandera as pa
from pandera import Column, DataFrameSchema, Check
from pandera.errors import SchemaError
from textwrap import fill

In [14]:
STANDARD_SCHEMA = {
'Indicator': ['indicator','Ind','مؤشر','المؤشر'],
'Country': ['الدولة','البلد','country'],
'Nationality': ['المواطنة','المواطنية','nationality','nationalities'],
'Sex': ['sex','gender','الجنس'],
'Year': ['year','time','Time','سنة','السنة'],
'Value': ['value','number','Number','الرقم','عدد', 'العدد'],
'Comments': ['comments','notes','ملاحظة','المصدر','Source'],
'Sector': ['sector','القطاع'],
'Education level': ['edu level','Education_level','المرحلة التعليمية','التعليم']
}


In [15]:
#map columns to col names using fuzzy match
col_names = {}
for col, aliases in STANDARD_SCHEMA.items():
    match, score, _ = process.extractOne(col, df_translated.columns.tolist() + aliases)
    
    if score > 80:
        print(f'{match} has a score of {score} for column {col}')
        col_names[match] = col

#rename column names according to schema
df_ar_translated=df_translated.rename(col_names)

df_ar_translated.columns

Ind has a score of 90.0 for column Indicator
country has a score of 85.71428571428572 for column Country
nationality has a score of 90.9090909090909 for column Nationality
Year has a score of 100.0 for column Year
Value has a score of 100.0 for column Value
comments has a score of 87.5 for column Comments
sector has a score of 83.33333333333334 for column Sector
Education_level has a score of 93.33333333333333 for column Education level


Index(['الدولة', 'المواطنة', 'الجنس', 'المؤشر', 'Year', 'Value', 'Source', 'القطاع', 'المرحلة التعليمية'], dtype='object')

#### validate the dataframe column labels

In [94]:
df_ar_translated.dtypes

Indicator           object
Country             object
Nationality         object
Sex                 object
Year                 int64
Value              float64
Source              object
Sector              object
Education level     object
dtype: object

In [28]:
translation_df=pd.read_excel('translation dict.xlsx')
translation_df.head(2)

,col_en,val_en,col_ar,val_ar
0,Indicator,Net enrolment rate in primary education (percent),المؤشر,معدل الإلتحاق الصافي في مرحلة التعليم الإبتدائي (الأساسي) (نسبة مئوية)
1,Indicator,Net enrolment rate in secondary education (percent),المؤشر,معدل الإلتحاق الصافي في مرحلة التعليم الثانوي (نسبة مئوية)


In [38]:
list(translation_df.loc[translation_df['col_en']=='Education level', 'val_en'].unique())

['Primary', 'Preparatory', 'Secondary', 'Tertiary', 'All levels']

In [ ]:
En_Ar_dictionary.keys()

In [43]:
allowed_indicator_values=list(translation_df.loc[translation_df['col_en']=='Indicator', 'val_en'].unique())
allowed_country_values=list(translation_df.loc[translation_df['col_en']=='Country', 'val_en'].unique())
allowed_nationality_values=list(translation_df.loc[translation_df['col_en']=='Nationality', 'val_en'].unique())
allowed_sex_values=list(translation_df.loc[translation_df['col_en']=='Sex', 'val_en'].unique())
allowed_year_values=[2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024]
allowed_sector_values=list(translation_df.loc[translation_df['col_en']=='Sector', 'val_en'].unique())
allowed_edulevel_values=list(translation_df.loc[translation_df['col_en']=='Education level', 'val_en'].unique())

#define schema
schema = DataFrameSchema({
    'Indicator': Column(str, checks=Check.isin(allowed_indicator_values)),
    'Country': Column(str, checks=Check.isin(allowed_country_values)),
    'Nationality': Column(str, checks=Check.isin(allowed_nationality_values),nullable=True),
    'Sex': Column(str, checks=Check.isin(allowed_sex_values),nullable=True),
    'Year': Column(int, checks=Check.isin(allowed_year_values)),
    'Sector': Column(str, checks=Check.isin(allowed_sector_values),nullable=True),
    'Education level': Column(str, checks=Check.isin(allowed_edulevel_values),nullable=True)     
})

#validate the dataframe
try:
    validated_df = schema.validate(df_ar_translated)
except SchemaError as err:
    wrapped_error = fill(str(err.args), width=150)
    print(wrapped_error)

("Column 'Indicator' failed element-wise validator number 0: isin(['Net enrolment rate in primary education (percent)', 'Net enrolment rate in
secondary education (percent)', 'Pupil-teacher ratio (percent)', 'Adult literacy rates', 'Youth literacy rates', 'Expenditure on education as a
percentage of total government expenditure  (percent)', 'Government expenditure on education as a percentage of GDP (percent)']) failure cases: wrong
ind",)


In [42]:
df_ar_translated.loc[0,'Indicator']='wrong ind'
df_ar_translated.head()

,Indicator,Country,Nationality,Sex,Year,Value,Source,Sector,Education level
0,wrong ind,Somalia,Nationality Total,Male,2010,1.0,a,NaN,NaN
1,Net enrolment rate in primary education (percent),Somalia,Nationality Total,NaN,2010,1.0,a,NaN,NaN
2,Net enrolment rate in primary education (percent),Somalia,Nationality Total,Both sexes,2010,1.0,a,NaN,NaN
3,Net enrolment rate in primary education (percent),Somalia,Nationals,Male,2010,1.0,a,NaN,NaN
4,Net enrolment rate in primary education (percent),Somalia,Nationals,NaN,2010,1.0,a,NaN,NaN


In [ ]:
# #for Algeria alone

# # Path to your service account JSON
# SERVICE_ACCOUNT_FILE = 'C:/Users/511232/Desktop/DSS/Consolidated questionnaires/online questionnnaires/credentials_serviceaccount.json'

# SCOPES = [
#     'https://www.googleapis.com/auth/drive.readonly',
#     'https://www.googleapis.com/auth/spreadsheets.readonly'
# ]

# FOLDER_ID = '1ftIKuPB_bnSKPiBNLblOtaz-11WQDUwq'  # Algeria folder ID

# # -----------------------------
# # AUTHENTICATE
# # -----------------------------
# creds = Credentials.from_service_account_file(
#     SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# drive_service = build('drive', 'v3', credentials=creds)
# sheets_service = build('sheets', 'v4', credentials=creds)

# # -----------------------------
# # LIST GOOGLE SHEETS IN ALGERIA
# # -----------------------------
# query = f"'{FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
# response = drive_service.files().list(q=query, fields="files(id, name)").execute()
# files = response.get('files', [])

# results = []

# keywords=['Health', 'Education','Housing', 'Population', 'Labor', 'Poverty']

# for file in files:
#     file_id = file['id']
#     file_name = file['name']

#     # Get sheet tabs
#     sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=file_id).execute()
#     sheets = sheet_metadata.get('sheets', [])

#     for sheet in sheets:
#         tab_name = sheet['properties']['title']

#         # Filter tabs with required keywords
#         if any(keyword.lower() in tab_name.lower() for keyword in keywords):
#             gid = sheet['properties']['sheetId']
#             url = f"https://docs.google.com/spreadsheets/d/{file_id}/edit#gid={gid}"

#             results.append({
#                 'file_name': file_name,
#                 'tab_name': tab_name,
#                 'gid': gid,
#                 'url': url
#             })

# # Display results
# df = pd.DataFrame(results)
# df.head()


In [ ]:
# # Path to your service account JSON
# SERVICE_ACCOUNT_FILE = 'C:/Users/511232/Desktop/DSS/Consolidated questionnaires/online questionnnaires/credentials_serviceaccount.json'

# # Define API scopes
# SCOPES = [
#     'https://www.googleapis.com/auth/drive.readonly',
#     'https://www.googleapis.com/auth/spreadsheets.readonly'
# ]

# # Authenticate and build the Drive and Sheets services
# creds = Credentials.from_service_account_file(
#     SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# # Initialize Drive and Sheets services
# drive_service = build('drive', 'v3', credentials=creds)
# sheets_service = build('sheets', 'v4', credentials=creds)
# #######################################################################################################

# # Set main folder ID
# MAIN_FOLDER_ID = '1nZ8l69GkB6EQQPoiOF-6CEVx2q3vAhcU'

# # List all country subfolders in main folder
# query = f"'{MAIN_FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.folder'"
# response = drive_service.files().list(q=query, fields="files(id, name)").execute()
# country_folders = response.get('files', [])

# results = []

# # Loop through each country folder
# keywords=['Health', 'Education','Housing', 'Population', 'Labor', 'Poverty']

# for folder in country_folders:
#     folder_id = folder['id']
#     folder_name = folder['name']
    
#     # List Google Sheets in the country folder
#     query_files = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'"
#     response_files = drive_service.files().list(q=query_files, fields="files(id, name)").execute()
#     files = response_files.get('files', [])

#     for file in files:
#         file_id = file['id']
#         file_name = file['name']
        
#         #Get sheet tabs and gids
#         sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=file_id).execute()
#         sheets = sheet_metadata.get('sheets', [])

#         for sheet in sheets:
#             tab_name = sheet['properties']['title']

#             if any(keyword.lower() in tab_name.lower() for keyword in keywords):

#                 gid = sheet['properties']['sheetId']
#                 url = f"https://docs.google.com/spreadsheets/d/{file_id}/edit#gid={gid}"

#                 results.append({
#                     'country_folder': folder_name,
#                     'file_name': file_name,
#                     'tab_name': tab_name,
#                     'gid': gid,
#                     'url': url
#                 })

# # Display results
# df_urls = pd.DataFrame(results)

# #make the url csv readable

# # df_urls.to_csv('C:/Users/511232/Desktop/DSS/Consolidated questionnaires/online questionnnaires/sheet_tabs_urls_all_countries.csv', index=False)
